In [20]:
import os
import pandas as pd
from tqdm import tqdm

datasets = ["Round1_T2D", "Round3", "2T_2020", "Round4", "HardTableR2", "HardTableR3"]


def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    ne_cols = {str(col_index): None for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{row[1]}-{row[2]}"] = qid
    return ne_cols, correct_qids


for dataset in datasets:
    cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
    tables = os.listdir(f"./Datasets/{dataset}/tables")
    for table in tqdm(tables, desc="Processing tables..."):
        if table.endswith(".csv"):
            df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
            table_name = table.split(".csv")[0]
            ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)    
    break

Processing tables...: 100%|██████████| 64/64 [00:00<00:00, 302.88it/s]


In [ ]:
ne_cols, correct_qids

In [5]:
import os
import pandas as pd
import requests
import json
from tqdm import tqdm

datasets = ["Round1_T2D", "Round3", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

# Function to get NE columns and correct QIDs from the GT file
def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    # Initialize ne_cols as a dict where the value (NER type) is None initially
    ne_cols = {str(col_index): None for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{row[1]}-{row[2]}"] = qid
    return ne_cols, correct_qids

# Function to call the NER API endpoint
def classify_columns_with_api(df, type="accurate"):
    payload = {
        "json": [df.T.astype(str).values.tolist()]  # Transpose to align columns and convert to list of rows
    }
    url = f'https://lamapi.hel.sintef.cloud/sti/column-analysis?model_type={type}&token=lamapi_demo_2023'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        return response.json()  # Return JSON response with NER classifications
    else:
        print(f"Error with API call: {response.status_code}")
        return None

# Main processing loop
for dataset in datasets:
    cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
    tables = os.listdir(f"./Datasets/{dataset}/tables")
    
    for table in tqdm(tables, desc="Processing tables..."):
        if table.endswith(".csv"):
            df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
            table_name = table.split(".csv")[0]

            # Get NE columns and correct QIDs from GT
            ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)
            
            # Call the NER API to classify columns
            ner_response = classify_columns_with_api(df)
            if ner_response:
                lit_cols = {}  # Initialize LIT columns dictionary
                for col_idx, ner_info in ner_response.items():
                    col_num = str(ner_info["index_column"])  # Get column index as string
                    classification = ner_info["classification"]  # Get NER or LIT type
                    tag = ner_info["tag"]

                    # Update NE columns from GT
                    if col_num in ne_cols:
                        # If GT classifies as NE, we trust the GT and update with the NER type
                        if tag == "LIT":
                            classification = "OTHER"
                        ne_cols[col_num] = classification
                    elif tag == "NE":
                        # If API classifies as NE and not in GT, add to ne_cols
                        ne_cols[col_num] = classification
                    elif tag == "LIT":
                        # If API classifies as LIT, add to lit_cols
                        lit_cols[col_num] = classification
        break
    break  # Keep the break for testing purposes

Processing tables...:   0%|          | 0/64 [00:06<?, ?it/s]


AttributeError: 'list' object has no attribute 'items'

In [7]:
ner_response

[{'table_1': {'0': {'index_column': 0,
    'tag': 'LIT',
    'classification': 'NUMBER',
    'datatype': 'NUMBER',
    'probabilities': {'NUMBER': 1.0}},
   '1': {'index_column': 1,
    'tag': 'NE',
    'classification': 'OTHER',
    'datatype': 'OTHER',
    'probabilities': {'LOCATION': 0.06,
     'PERSON': 0.02,
     'OTHER': 0.84,
     'STRING': 0.06}},
   '2': {'index_column': 2,
    'tag': 'LIT',
    'classification': 'NUMBER',
    'datatype': 'NUMBER',
    'probabilities': {'NUMBER': 1.0, 'DATE': 1.0}},
   '3': {'index_column': 3,
    'tag': 'NE',
    'classification': 'PERSON',
    'datatype': 'PERSON',
    'probabilities': {'PERSON': 1.0}},
   '4': {'index_column': 4,
    'tag': 'LIT',
    'classification': 'NUMBER',
    'datatype': 'NUMBER',
    'probabilities': {'NUMBER': 1.0}}}}]

In [6]:
ne_cols, lit_cols, tag, ner_response

NameError: name 'tag' is not defined

In [55]:
df

,col0,col1,col2
0,Somosierra,Autovía A-1,Spain
1,Fotu La,National Highway 1,India
2,Zojila Pass,National Highway 1,India
3,Jablunkov Pass,European route E75,Czech Republic
4,Wildhaus Pass,Main road 16,Switzerland
5,Pacific Grade Summit,California State Route 4,United States of America
6,Mahoosuc Notch,Appalachian Trail,United States of America
7,Zealand Notch,Appalachian Trail,United States of America
8,Summit Pass,Alaska Highway,Canada
9,Muncho Pass,Alaska Highway,Canada


In [4]:
import os
import pandas as pd
import requests
import json
from pymongo import MongoClient, ASCENDING
from tqdm import tqdm

# MongoDB connection
client = MongoClient("mongodb://mongodb:27017/")
db = client["crocodile_db"]
input_collection = db["input_data"]
table_trace_collection = db["table_trace"]
dataset_trace_collection = db["dataset_trace"]
process_queue = db["process_queue"]

# Ensure indexes for uniqueness and performance
input_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING), ("row_id", ASCENDING)], unique=True)
table_trace_collection.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)
dataset_trace_collection.create_index([("dataset_name", ASCENDING)], unique=True)
process_queue.create_index([("dataset_name", ASCENDING), ("table_name", ASCENDING)], unique=True)

datasets = ["Round1_T2D", "Round3", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

# Function to get NE columns and correct QIDs from the GT file
def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    # Initialize ne_cols as a dict where the value (NER type) is None initially
    ne_cols = {str(col_index): None for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{row[1]}-{row[2]}"] = qid
    return ne_cols, correct_qids

# Function to call the NER API endpoint
def classify_columns_with_api(df, type="accurate"):
    payload = {
        "json": df.T.astype(str).values.tolist()  # Transpose to align columns and convert to list of rows
    }
    url = f'https://lamapi.hel.sintef.cloud/sti/column-analysis?model_type={type}&token=lamapi_demo_2023'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        return response.json()  # Return JSON response with NER classifications
    else:
        print(f"Error with API call: {response.status_code}")
        return None

# Function to onboard data into MongoDB
def onboard_data(dataset_name, table_name, df, ne_cols, correct_qids, ner_response):
    lit_cols = {}
    
    # Parse the API response for NE and LIT columns
    if ner_response:
        for col_idx, ner_info in ner_response.items():
            col_num = str(ner_info["index_column"])
            classification = ner_info["classification"]
            tag = ner_info["tag"]
            
            if col_num in ne_cols:
                # If GT classifies as NE, we trust the GT and update with NER type
                if tag == "LIT":
                    classification = "OTHER"
                ne_cols[col_num] = classification
            elif tag == "NE":
                # If API classifies as NE but it's not in GT, add it to NE columns
                ne_cols[col_num] = classification
            elif tag == "LIT":
                # If API classifies as LIT, add to LIT columns
                lit_cols[col_num] = classification

    # Insert data row by row
    for index, row in df.iterrows():
        document = {
            "dataset_name": dataset_name,
            "table_name": table_name,
            "row_id": index,
            "data": row.to_dict(),
            "classified_columns": {
                "NE": ne_cols,  # Use updated NE columns
                "LIT": lit_cols  # Use LIT columns from the API response
            },
            "context_columns": [str(i) for i in range(len(df.columns))],  # Context columns
            "correct_qids": correct_qids,  # Correct QIDs from GT
            "status": "TODO"
        }

        # Insert or update the document in MongoDB
        input_collection.update_one(
            {"dataset_name": dataset_name, "table_name": table_name, "row_id": index},
            {"$set": document},
            upsert=True
        )

    # Log onboarding completion for table-level trace
    table_trace_collection.update_one(
        {"dataset_name": dataset_name, "table_name": table_name},
        {"$set": {
            "total_rows": len(df),
            "processed_rows": 0,
            "status": "PENDING"
        }},
        upsert=True
    )

# Main processing loop for onboarding datasets
for dataset in datasets:
    cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
    tables = os.listdir(f"./Datasets/{dataset}/tables")
    
    for table in tqdm(tables, desc=f"Processing tables for dataset {dataset}..."):
        if table.endswith(".csv"):
            df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
            table_name = table.split(".csv")[0]

            # Get NE columns and correct QIDs from GT
            ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)
            
            # Call the NER API to classify columns
            ner_response = classify_columns_with_api(df, type="fast")

            # Onboard the data into MongoDB
            onboard_data(dataset, table_name, df, ne_cols, correct_qids, ner_response)

    # Initialize dataset-level trace after onboarding all tables in the dataset
    dataset_trace_collection.update_one(
        {"dataset_name": dataset},
        {"$setOnInsert": {
            "total_tables": len(tables),
            "processed_tables": 0,
            "total_rows": 0,  # Updated as tables are processed
            "processed_rows": 0,
            "status": "PENDING"
        }},
        upsert=True
    )
    break

Processing tables for dataset Round1_T2D...: 100%|██████████| 64/64 [00:50<00:00,  1.28it/s]


In [8]:
import os
import pandas as pd
import requests
import json
from tqdm import tqdm

datasets = ["Round1_T2D", "Round3", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

# Function to get NE columns and correct QIDs from the GT file
def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    # Initialize ne_cols as a dict where the value (NER type) is None initially
    ne_cols = {str(col_index): None for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{row[1]}-{row[2]}"] = qid
    return ne_cols, correct_qids

# Function to call the NER API endpoint with multiple tables
def classify_columns_with_api(tables_data, type="fast"):
    payload = {
        "json": [df.T.astype(str).values.tolist() for df in tables_data]  # Multiple tables as input
    }
    url = f'https://lamapi.hel.sintef.cloud/sti/column-analysis?model_type={type}&token=lamapi_demo_2023'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        return response.json()  # Return JSON response with NER classifications
    else:
        print(f"Error with API call: {response.status_code}")
        return None

# Main processing loop
def process_tables(datasets, max_tables_at_once=1):
    for dataset in datasets:
        cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
        tables = os.listdir(f"./Datasets/{dataset}/tables")
        tables_data = []
        processed_count = 0

        for table in tqdm(tables, desc="Processing tables..."):
            if table.endswith(".csv"):
                df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
                table_name = table.split(".csv")[0]

                # Get NE columns and correct QIDs from GT
                ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)
                
                # Add table data to list
                tables_data.append(df)
                processed_count += 1

                # If we reach the maximum number of tables, make the API call
                if processed_count >= max_tables_at_once:
                    # Call the NER API to classify columns for these tables
                    ner_response = classify_columns_with_api(tables_data)
                    
                    if ner_response:
                        for table_response in ner_response:
                            table_key = list(table_response.keys())[0]
                            lit_cols = {}  # Initialize LIT columns dictionary
                            for col_idx, ner_info in table_response[table_key].items():
                                col_num = str(ner_info["index_column"])  # Get column index as string
                                classification = ner_info["classification"]  # Get NER or LIT type
                                tag = ner_info["tag"]

                                # Update NE columns from GT
                                if col_num in ne_cols:
                                    # If GT classifies as NE, we trust the GT and update with the NER type
                                    if tag == "LIT":
                                        classification = "OTHER"
                                    ne_cols[col_num] = classification
                                elif tag == "NE":
                                    # If API classifies as NE and not in GT, add to ne_cols
                                    ne_cols[col_num] = classification
                                elif tag == "LIT":
                                    # If API classifies as LIT, add to lit_cols
                                    lit_cols[col_num] = classification

                    # Reset tables data for the next batch
                    tables_data = []
                    processed_count = 0
        
        # In case there are remaining tables that haven't been processed yet
        if tables_data:
            ner_response = classify_columns_with_api(tables_data)
        return

# Example of running the function
process_tables(datasets, max_tables_at_once=10)  # Adjust the number of tables you want to submit at once

Processing tables...: 100%|██████████| 64/64 [01:28<00:00,  1.38s/it]


In [9]:
import os
import pandas as pd
import requests
import json
from tqdm import tqdm

datasets = ["Round1_T2D", "Round3", "2T_2020", "Round4_2020", "HardTablesR2", "HardTablesR3"]

# Function to get NE columns and correct QIDs from the GT file
def get_ne_cols_and_correct_qids(table_name, cea_gt):
    correct_qids = {}
    filtered_cea_gt = cea_gt[cea_gt[0] == table_name]
    # Initialize ne_cols as a dict where the value (NER type) is None initially
    ne_cols = {str(col_index): None for col_index in filtered_cea_gt[2].unique()}
    for _, row in filtered_cea_gt.iterrows():
        qid = row[3].split("/")[-1]
        correct_qids[f"{row[1]}-{row[2]}"] = qid
    return ne_cols, correct_qids

# Function to call the NER API endpoint with multiple tables
def classify_columns_with_api(tables_data, type="fast"):
    payload = {
        "json": [df.T.astype(str).values.tolist() for df in tables_data]  # Multiple tables as input
    }
    url = f'https://lamapi.hel.sintef.cloud/sti/column-analysis?model_type={type}&token=lamapi_demo_2023'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        return response.json()  # Return JSON response with NER classifications
    else:
        print(f"Error with API call: {response.status_code}")
        return None

# Main processing loop
def process_tables(datasets, max_tables_at_once=1):
    for dataset in datasets:
        cea_gt = pd.read_csv(f"./Datasets/{dataset}/gt/cea.csv", header=None)
        tables = os.listdir(f"./Datasets/{dataset}/tables")
        tables_data = []
        processed_count = 0

        for table in tqdm(tables, desc="Processing tables..."):
            if table.endswith(".csv"):
                df = pd.read_csv(f"./Datasets/{dataset}/tables/{table}")
                table_name = table.split(".csv")[0]

                # Get NE columns and correct QIDs from GT
                ne_cols, correct_qids = get_ne_cols_and_correct_qids(table_name, cea_gt)
                
                # Add table data to list
                tables_data.append(df)
                processed_count += 1

                # If we reach the maximum number of tables, make the API call
                if processed_count >= max_tables_at_once:
                    # Call the NER API to classify columns for these tables
                    ner_response = classify_columns_with_api(tables_data)
                    
                    if ner_response:
                        for table_response in ner_response:
                            table_key = list(table_response.keys())[0]
                            lit_cols = {}  # Initialize LIT columns dictionary
                            for col_idx, ner_info in table_response[table_key].items():
                                col_num = str(ner_info["index_column"])  # Get column index as string
                                classification = ner_info["classification"]  # Get NER or LIT type
                                tag = ner_info["tag"]

                                # Update NE columns from GT
                                if col_num in ne_cols:
                                    # If GT classifies as NE, we trust the GT and update with the NER type
                                    if tag == "LIT":
                                        classification = "OTHER"
                                    ne_cols[col_num] = classification
                                elif tag == "NE":
                                    # If API classifies as NE and not in GT, add to ne_cols
                                    ne_cols[col_num] = classification
                                elif tag == "LIT":
                                    # If API classifies as LIT, add to lit_cols
                                    lit_cols[col_num] = classification

                    # Reset tables data for the next batch
                    tables_data = []
                    processed_count = 0
        
        # In case there are remaining tables that haven't been processed yet
        if tables_data:
            ner_response = classify_columns_with_api(tables_data)
        return

# Example of running the function
process_tables(datasets, max_tables_at_once=10)  # Adjust the number of tables you want to submit at once

Processing tables...: 100%|██████████| 64/64 [00:07<00:00,  8.17it/s]
